# 美国国债收益率曲线构建与分析演示

本Notebook演示了如何使用yield-curve-construction工具包完成以下任务：
1. 数据加载和预处理
2. 构建收益率曲线
3. 曲线插值和可视化
4. 债券定价分析
5. Z-Spread计算

In [ ]:
import sys
import os
sys.path.append(os.path.join('..', '..'))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# 导入项目模块
from curves.bootstrapping.bootstrapper import bootstrap_curve_from_dataframe
from curves.instruments.factory import InstrumentFactory
from curves.curve import YieldCurve
from curves.interpolation.cubic_spline import CubicSplineInterpolator
from curves.interpolation.nelson_siegel_svensson import NelsonSiegelSvenssonInterpolator
from pricing import BondPricer
from pricing.z_spread import solve_z_spread, price_with_z_spread

print("所有模块导入成功！")

## 1. 数据加载和预处理

In [ ]:
# 加载数据
data_path = '../data/treasuries_2018-12-28.parquet'
print(f"正在加载数据: {data_path}")

df = pd.read_parquet(data_path)
print(f"数据形状: {df.shape}")
print(f"数据列: {list(df.columns)}")
print("\n前5行数据:")
df.head()

In [ ]:
# 数据基本信息
print("数据基本信息:")
print(f"交易日期范围: {df['CALDT'].min()} 到 {df['CALDT'].max()}")
print(f"工具类型分布:")
print(df['ITYPE'].value_counts())
print(f"\nCUSIP数量: {df['TCUSIP'].nunique()}")
print(f"到期日期范围: {df['TMATDT'].min()} 到 {df['TMATDT'].max()}")

In [ ]:
# 选择特定日期的数据
target_date = '2018-12-28'
day_df = df[df['CALDT'] == target_date].copy()
print(f"{target_date} 的数据行数: {len(day_df)}")

# 按到期日期排序
day_df = day_df.sort_values('TMATDT')
print("\n按到期日期排序后的前10行:")
day_df[['TCUSIP', 'CALDT', 'TMATDT', 'ITYPE', 'TDNOMPRC', 'TDACCINT']].head(10)

## 2. 金融工具转换

In [ ]:
# 将数据转换为instrument对象
print("正在转换数据为金融工具对象...")
instruments, errors = InstrumentFactory.from_dataframe(day_df)

print(f"成功转换: {len(instruments)} 个工具")
print(f"转换错误: {len(errors)} 个")

if errors:
    print("\n前5个错误:")
    for i, error in enumerate(errors[:5]):
        print(f"  {i+1}. {error}")

In [ ]:
# 分析工具类型
instrument_types = {}
for inst in instruments:
    inst_type = type(inst).__name__
    if inst_type not in instrument_types:
        instrument_types[inst_type] = []
    instrument_types[inst_type].append(inst)

print("工具类型分布:")
for inst_type, inst_list in instrument_types.items():
    print(f"  {inst_type}: {len(inst_list)} 个")
    print(f"    到期日期范围: {min(inst.maturity_date for inst in inst_list)} 到 {max(inst.maturity_date for inst in inst_list)}")

## 3. 构建收益率曲线

In [ ]:
# 使用引导算法构建收益率曲线
print("正在构建收益率曲线...")

curve, nodes, interpolator, errors = bootstrap_curve_from_dataframe(
    day_df,
    interpolator='loglinear',  # 使用对数线性插值
    use_liquidity_tiebreak=True
)

print(f"\n构建结果:")
print(f"  估值日: {curve.val_date}")
print(f"  节点数量: {len(nodes)}")
print(f"  插值器类型: {type(interpolator).__name__}")
print(f"  转换错误: {len(errors)} 个")

if errors:
    print("\n前5个错误:")
    for i, error in enumerate(errors[:5]):
        print(f"  {i+1}. {error}")

In [ ]:
# 查看节点信息
print("收益率曲线节点:")
print("时间(年)\t折现因子\t零息收益率(%)\t即期利率(%)")
print("-" * 60)

for t, df in nodes:
    zero_rate = -np.log(df) / t * 100 if t > 0 else 0
    spot_rate = (1/df - 1) / t * 100 if t > 0 else 0
    print(f"{t:.2f}\t\t{df:.6f}\t{zero_rate:.4f}\t\t{spot_rate:.4f}")

## 4. 曲线插值和可视化

In [ ]:
# 生成平滑的收益率曲线
max_t = max(t for t, _ in nodes)
t_values = np.linspace(0.01, max_t, 1000)

# 计算各种利率
zero_rates = [curve.zero_rate_cc(t) * 100 for t in t_values]
spot_rates = [curve.zero_rate_simple(t) * 100 for t in t_values]
discount_factors = [curve.df(t) for t in t_values]

# 绘制收益率曲线
plt.figure(figsize=(14, 10))

# 子图1: 零息收益率曲线
plt.subplot(2, 2, 1)
plt.plot(t_values, zero_rates, 'b-', linewidth=2, label='零息收益率')
plt.scatter([t for t, _ in nodes], [-np.log(df)/t*100 for t, df in nodes], 
           c='red', s=50, zorder=5, label='原始节点')
plt.xlabel('期限 (年)')
plt.ylabel('收益率 (%)')
plt.title('零息收益率曲线')
plt.grid(True, alpha=0.3)
plt.legend()

# 子图2: 即期利率曲线
plt.subplot(2, 2, 2)
plt.plot(t_values, spot_rates, 'g-', linewidth=2, label='即期利率')
plt.scatter([t for t, _ in nodes], [(1/df-1)/t*100 for t, df in nodes], 
           c='red', s=50, zorder=5, label='原始节点')
plt.xlabel('期限 (年)')
plt.ylabel('利率 (%)')
plt.title('即期利率曲线')
plt.grid(True, alpha=0.3)
plt.legend()

# 子图3: 折现因子曲线
plt.subplot(2, 2, 3)
plt.plot(t_values, discount_factors, 'orange', linewidth=2, label='折现因子')
plt.scatter([t for t, _ in nodes], [df for _, df in nodes], 
           c='red', s=50, zorder=5, label='原始节点')
plt.xlabel('期限 (年)')
plt.ylabel('折现因子')
plt.title('折现因子曲线')
plt.grid(True, alpha=0.3)
plt.legend()

# 子图4: 远期利率曲线
forward_rates = []
forward_t = []
for i in range(len(t_values)-1):
    t1, t2 = t_values[i], t_values[i+1]
    df1, df2 = curve.df(t1), curve.df(t2)
    if df1 > 0 and df2 > 0:
        f_rate = (np.log(df1) - np.log(df2)) / (t2 - t1) * 100
        forward_rates.append(f_rate)
        forward_t.append((t1 + t2) / 2)

plt.subplot(2, 2, 4)
plt.plot(forward_t, forward_rates, 'purple', linewidth=2, label='远期利率')
plt.xlabel('期限 (年)')
plt.ylabel('远期利率 (%)')
plt.title('远期利率曲线')
plt.grid(True, alpha=0.3)
plt.legend()

plt.tight_layout()
plt.savefig('../data/yield_curve_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

## 5. 不同插值方法比较

In [ ]:
# 比较不同的插值方法
print("比较不同插值方法...")

# 对数线性插值（已构建）
loglinear_curve = curve

# 三次样条插值
spline_interpolator = CubicSplineInterpolator()
spline_curve = YieldCurve(val_date=curve.val_date, nodes=nodes)
spline_curve.fit(spline_interpolator)

# Nelson-Siegel-Svensson插值
nss_interpolator = NelsonSiegelSvenssonInterpolator()
nss_curve = YieldCurve(val_date=curve.val_date, nodes=nodes)
nss_curve.fit(nss_interpolator)

print("插值方法构建完成！")

In [ ]:
# 绘制不同插值方法的比较
plt.figure(figsize=(15, 5))

methods = [
    ('对数线性', loglinear_curve, 'blue'),
    ('三次样条', spline_curve, 'red'),
    ('Nelson-Siegel-Svensson', nss_curve, 'green')
]

for i, (name, curve_obj, color) in enumerate(methods):
    plt.subplot(1, 3, i+1)
    rates = [curve_obj.zero_rate_cc(t) * 100 for t in t_values]
    plt.plot(t_values, rates, color=color, linewidth=2, label=name)
    plt.scatter([t for t, _ in nodes], [-np.log(df)/t*100 for t, df in nodes], 
               c='black', s=30, zorder=5, label='原始节点')
    plt.xlabel('期限 (年)')
    plt.ylabel('零息收益率 (%)')
    plt.title(f'{name}插值')
    plt.grid(True, alpha=0.3)
    plt.legend()

plt.tight_layout()
plt.savefig('../data/interpolation_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

## 6. 债券定价分析

In [ ]:
# 选择几个不同类型的债券进行定价分析
print("选择债券进行定价分析...")

# 选择3个不同类型的债券
selected_bonds = []
for inst in instruments:
    if len(selected_bonds) >= 3:
        break
    if hasattr(inst, 'coupon_rate') and inst.coupon_rate > 0:
        selected_bonds.append(inst)

# 如果没有付息债券，选择短期国债
if len(selected_bonds) < 3:
    for inst in instruments:
        if len(selected_bonds) >= 3:
            break
        if inst.__class__.__name__ == 'Bill':
            selected_bonds.append(inst)

print(f"选择了 {len(selected_bonds)} 个债券进行分析")

In [ ]:
# 使用定价器为债券定价
pricer = BondPricer(loglinear_curve)

print("债券定价分析结果:\n")
print("=" * 80)

for i, bond in enumerate(selected_bonds, 1):
    print(f"债券 {i}: {type(bond).__name__}")
    print(f"  CUSIP: {bond.cusip}")
    print(f"  到期日: {bond.maturity_date}")
    print(f"  票息率: {getattr(bond, 'coupon_rate', 0)*100:.2f}%")
    print(f"  付息频率: {getattr(bond, 'freq', 'N/A')}")
    print(f"  市场价格: {bond.clean_price:.4f}")
    print(f"  应计利息: {bond.accrued_interest:.4f}")
    
    # 计算理论价格
    result = pricer.price(bond)
    
    print(f"\n  定价结果:")
    print(f"    理论Dirty Price: {result.dirty_price:.4f}")
    print(f"    理论Clean Price: {result.clean_price:.4f}")
    print(f"    价格差异: {result.dirty_price - bond.dirty_price:.4f}")
    
    # 现金流分析
    print(f"\n  现金流分析:")
    for j, cf in enumerate(result.cashflow_pvs[:5]):  # 只显示前5个
        print(f"    {cf.pay_date.strftime('%Y-%m-%d')}: ${cf.amount:.2f}, t={cf.t:.2f}, PV={cf.pv:.2f}")
    if len(result.cashflow_pvs) > 5:
        print(f"    ... 还有 {len(result.cashflow_pvs)-5} 个现金流")
    
    print("-" * 80)

## 7. Z-Spread计算

In [ ]:
# 计算Z-Spread
print("计算Z-Spread...\n")
print("=" * 80)

for i, bond in enumerate(selected_bonds, 1):
    print(f"债券 {i}: {type(bond).__name__} ({bond.cusip})")
    
    try:
        # 计算Z-Spread
        z_spread = solve_z_spread(
            instrument=bond,
            curve=loglinear_curve,
            target_dirty_price=bond.dirty_price
        )
        
        print(f"  Z-Spread: {z_spread * 10000:.2f} bps")
        
        # 使用Z-Spread重新定价
        price_with_spread = price_with_z_spread(
            instrument=bond,
            curve=loglinear_curve,
            spread=z_spread
        )
        
        print(f"  使用Z-Spread定价: {price_with_spread:.4f}")
        print(f"  与市场价格差异: {price_with_spread - bond.dirty_price:.6f}")
        
    except Exception as e:
        print(f"  Z-Spread计算失败: {e}")
    
    print("-" * 80)

## 8. 敏感性分析

In [ ]:
# 分析收益率曲线对不同参数的敏感性
print("收益率曲线敏感性分析\n")

# 选择一个基准债券
benchmark_bond = selected_bonds[0]
print(f"基准债券: {type(benchmark_bond).__name__} ({benchmark_bond.cusip})")
print(f"  到期日: {benchmark_bond.maturity_date}")
print(f"  票息率: {getattr(benchmark_bond, 'coupon_rate', 0)*100:.2f}%")

# 计算不同收益率水平下的价格
yield_shifts = np.linspace(-0.02, 0.02, 21)  # -200bps 到 +200bps
prices = []

for shift in yield_shifts:
    # 创建平移后的曲线
    shifted_curve = YieldCurve(
        val_date=curve.val_date,
        nodes=[(t, df * np.exp(-shift * t)) for t, df in nodes]
    )
    
    # 计算价格
    shifted_pricer = BondPricer(shifted_curve)
    result = shifted_pricer.price(benchmark_bond)
    prices.append(result.dirty_price)

# 绘制价格-收益率关系
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(yield_shifts * 100, prices, 'b-', linewidth=2)
plt.axhline(y=benchmark_bond.dirty_price, color='r', linestyle='--', label='市场价格')
plt.axvline(x=0, color='gray', linestyle='-', alpha=0.3)
plt.xlabel('收益率变化 (bps)')
plt.ylabel('债券价格')
plt.title('价格-收益率关系')
plt.grid(True, alpha=0.3)
plt.legend()

# 计算久期（近似）
prices_array = np.array(prices)
yields_array = yield_shifts

# 使用中心差分计算久期
dP_dy = np.gradient(prices_array, yields_array)
duration = -dP_dy / benchmark_bond.dirty_price

plt.subplot(1, 2, 2)
plt.plot(yield_shifts * 100, duration, 'g-', linewidth=2)
plt.axvline(x=0, color='gray', linestyle='-', alpha=0.3)
plt.xlabel('收益率变化 (bps)')
plt.ylabel('修正久期')
plt.title('久期分析')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../data/sensitivity_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\n基准债券久期（0收益率变化时）: {duration[len(duration)//2]:.2f}")

## 9. 总结

In [ ]:
print("=" * 80)
print("收益率曲线构建与分析总结")
print("=" * 80)

print(f"\n📊 数据概况:")
print(f"  数据日期: {target_date}")
print(f"  总记录数: {len(day_df)}")
print(f"  成功转换: {len(instruments)} 个金融工具")

print(f"\n📈 收益率曲线:")
print(f"  估值日: {curve.val_date}")
print(f"  节点数量: {len(nodes)}")
print(f"  期限范围: {min(t for t, _ in nodes):.2f} - {max(t for t, _ in nodes):.2f} 年")
print(f"  最低收益率: {min(-np.log(df)/t*100 for t, df in nodes):.2f}%")
print(f"  最高收益率: {max(-np.log(df)/t*100 for t, df in nodes):.2f}%")

print(f"\n💰 定价分析:")
print(f"  分析债券数: {len(selected_bonds)}")
for i, bond in enumerate(selected_bonds, 1):
    result = pricer.price(bond)
    diff = result.dirty_price - bond.dirty_price
    print(f"    债券 {i}: 价格差异 = {diff:.4f}")

print(f"\n🔧 使用的插值方法:")
print(f"  - 对数线性插值")
print(f"  - 三次样条插值")
print(f"  - Nelson-Siegel-Svensson模型")

print(f"\n📁 生成的文件:")
print(f"  - ../data/yield_curve_analysis.png (收益率曲线分析图)")
print(f"  - ../data/interpolation_comparison.png (插值方法比较图)")
print(f"  - ../data/sensitivity_analysis.png (敏感性分析图)")

print("\n✅ 演示完成！")

## 附录：常用操作

In [ ]:
# 一些常用的便捷操作

def get_yield_at_date(curve, target_date):
    """获取指定日期的收益率"""
    from curves.bootstrapping.daycount import yearfrac
    t = yearfrac(curve.val_date, target_date)
    return curve.zero_rate_cc(t)

def calculate_forward_rate(curve, start_date, end_date):
    """计算远期利率"""
    from curves.bootstrapping.daycount import yearfrac
    t1 = yearfrac(curve.val_date, start_date)
    t2 = yearfrac(curve.val_date, end_date)
    df1 = curve.df(t1)
    df2 = curve.df(t2)
    if df1 > 0 and df2 > 0:
        return (np.log(df1) - np.log(df2)) / (t2 - t1)
    return None

# 示例：计算1年后的1年期远期利率
val_date = curve.val_date
start_date = val_date + timedelta(days=365)
end_date = val_date + timedelta(days=730)

forward_rate = calculate_forward_rate(curve, start_date, end_date)
if forward_rate:
    print(f"1年后的1年期远期利率: {forward_rate*100:.2f}%")

# 示例：获取不同期限的收益率
tenors = [0.5, 1, 2, 5, 10, 20, 30]
print("\n不同期限的收益率:")
for t in tenors:
    rate = curve.zero_rate_cc(t)
    print(f"  {t:2.0f}年: {rate*100:5.2f}%")